# 决策树建模
对于sklearn中的决策树，参看[DecisionTree](https://github.com/xuhaer/ML/blob/master/Code/Day%2025_Decision_Tree.ipynb)

针对某个web站点的用户行为及其最终购买决策：
列: 来源网站，位置，是否阅读过FAQ，浏览网页数，最终购买类型

In [1]:
my_data = [['slashdot', 'USA', 'yes', 18, 'None'],
               ['google', 'France', 'yes', 23, 'Premium'],
               ['digg', 'USA', 'yes', 24, 'Basic'],
               ['kiwitobes', 'France', 'yes', 23, 'Basic'],
               ['google', 'UK', 'no', 21, 'Premium'],
               ['(direct)', 'New Zealand', 'no', 12, 'None'],
               ['(direct)', 'UK', 'no', 21, 'Basic'],
               ['google', 'USA', 'no', 24, 'Premium'],
               ['slashdot', 'France', 'yes', 19, 'None'],
               ['digg', 'USA', 'no', 18, 'None'],
               ['google', 'UK', 'no', 18, 'None'],
               ['kiwitobes', 'UK', 'no', 19, 'None'],
               ['digg', 'New Zealand', 'yes', 12, 'Basic'],
               ['slashdot', 'UK', 'no', 21, 'None'],
               ['google', 'UK', 'yes', 18, 'Basic'],
               ['kiwitobes', 'France', 'yes', 19, 'Basic']]

### 引入决策树：
* col是待检测的判断条件所对应的列索引值。
* value对应于为了使结果为true,当前列必须匹配的值。
* tb、fb为decisionnode，对应与true或false时的子树
* results保存的是针对于当前分支的结果，是一个字典。除叶节点外，在其它节点上该值均为None

In [2]:
class decisionnode:
    def __init__(self, col=-1, value=None, results=None, tb=None, fb=None):
        self.col = col
        self.value = value
        self.results = results
        self.tb = tb
        self.fb = fb


divideset的作用是根据列表中某一栏的数据将列表拆分为两个数据集。

In [3]:
def divideset(rows, column, value):
    # Make a function that tells us if a row is in
    # the first group (true) or the second group (false)
    split_function = None
    if isinstance(value, int) or isinstance(value, float):
            split_function = lambda row: row[column] >= value
    else:
            split_function = lambda row: row[column] == value
    # Divide the rows into two sets and return them
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return (set1, set2)

比如根据第二列的值是否为'yes'将my_data拆分为2个集合:

In [4]:
divideset(my_data, 2, 'yes')

([['slashdot', 'USA', 'yes', 18, 'None'],
  ['google', 'France', 'yes', 23, 'Premium'],
  ['digg', 'USA', 'yes', 24, 'Basic'],
  ['kiwitobes', 'France', 'yes', 23, 'Basic'],
  ['slashdot', 'France', 'yes', 19, 'None'],
  ['digg', 'New Zealand', 'yes', 12, 'Basic'],
  ['google', 'UK', 'yes', 18, 'Basic'],
  ['kiwitobes', 'France', 'yes', 19, 'Basic']],
 [['google', 'UK', 'no', 21, 'Premium'],
  ['(direct)', 'New Zealand', 'no', 12, 'None'],
  ['(direct)', 'UK', 'no', 21, 'Basic'],
  ['google', 'USA', 'no', 24, 'Premium'],
  ['digg', 'USA', 'no', 18, 'None'],
  ['google', 'UK', 'no', 18, 'None'],
  ['kiwitobes', 'UK', 'no', 19, 'None'],
  ['slashdot', 'UK', 'no', 21, 'None']])

假设第二列是否为'yes'与用户最终购买服务有直接关系， 基于上面的结果可知：yes: 2 None, 1 Pre, 5 Basic, no: 5 None, 2 Pre, 1 Basic

结果并非很理想，而且，也不直观，因此我们需要引入一种方法来“衡量”其“理想化”

In [5]:
# 对各种可能的结果进行计数，返回一个字典，其中包含了每一项结果的出现次数，为接下来计算数据集合的复杂度函数提供依据。
def uniquecounts(rows):
    results = {}
    for row in rows:
            # The result is the last column
            r = row[len(row)-1]
            if r not in results: results[r] = 0
            results[r] += 1
    return results

对于混杂程度的测试，有几种不同的度量方式可供选择，此处我们将考察其中的两种：<font color='red'>**基尼不纯度(Gini impurity) 和 熵(entropy)** <font>

### **基尼不纯度**：是指将来自集合中的某种结果随机应用于集合中的某一数据项的预期误差率。
<br>
基尼不纯度的计算函数如下：

In [6]:
def giniimpurity(rows):
    total = len(rows)
    results = uniquecounts(rows)
    imp = 0
    for k1 in results:
        p1 = float(results[k1]) / total
        for k2 in results:
            if k1 == k2: continue
            p2 = float(results[k2]) / total
            imp += p1*p2
    return imp

### **熵**: 在信息理论中，熵代表的是集合的无序程度--基本上就相当于我们在此处所说的集合的混杂程度。

如果所有结果都相同(比如此例中，不论怎样，最终所有人都成为了付费用户),则熵为0。群组越是混乱，则其熵就越高。

In [7]:
# 熵是遍历所有可能的结果之后所得到的p(x)*log(p(x))之和
def entropy(rows):
    from math import log
    log2 = lambda x: log(x)/log(2) #换底公式
    results = uniquecounts(rows)
    # Now calculate the entropy
    ent = 0.0
    for r in results.keys():
            p = float(results[r])/len(rows)
            ent = ent-p*log2(p)
    return ent

In [8]:
 giniimpurity(my_data)

0.6328125

In [9]:
entropy(my_data)

1.5052408149441479

In [10]:
set1, set2 = divideset(my_data, 2, 'yes')

看看根据第二列是否阅读FAQ拆分后的数据集的混杂程度：

In [11]:
giniimpurity(set1)

0.53125

In [12]:
entropy(set1)

1.2987949406953985

可以看到，基尼不纯度减少约0.1，熵减少约0.2， 熵和基尼不纯度的主要区别在于：熵达到峰值的过程要相对慢一些。因此，熵对于混乱集合的‘判罚’要更重一些。 由于人们对熵的使用更为普遍，此后的将选择熵作为度量标准

### 以递归的方式构造树
为了弄明白一个属性的好坏程度，我们的算法是首先求出整个群组的熵，然后尝试利用每个属性的可能取值对群组进行拆分，并求出两个新群组的熵。为了确定哪个属性最合适用来拆分，算法会计算相应的<font color='red'> **信息增益(information gain)** </font>, 所谓信息增益，指当前熵与两个新群组经加权后的熵之间的差值。算法会针对每个属性计算相应的信息增益，然后从中选出信息增益最大的属性。

In [13]:
def buildtree(rows, scoref=entropy):
    '''接受一个由数据行构成的列表作为参数。遍历了数据集中的每一列(最后一列除外，那是同来存放最终结果的)，
    针对各列查找的每一列可能的取值，并将数据集拆分为两个新的子集。通过将子集的熵乘以子集中所含数据项在
    原数据集中所占的比重(fraction)，函数求出了每一列对新生成的子集的加权平均熵，并记录下熵值最低的那一对子集。
    如果由熵值最低的一对子集求的的加权平均熵比当前集合的熵大，拆分结束。否则，递归调用，并把调用的结果添加到树上。'''
    if len(rows)==0:
        return decisionnode()
    current_score=scoref(rows)

    # Set up some variables to track the best criteria
    best_gain=0.0
    best_criteria=None
    best_sets=None
    
    column_count=len(rows[0])-1
    for col in range(0,column_count):
        # Generate the list of different values in
        # this column
        column_values={}
        for row in rows:
             column_values[row[col]]=1
        # Now try dividing the rows up for each value
        # in this column
        for value in column_values.keys():
            (set1,set2)=divideset(rows,col,value)
            
            # Information gain
            # 信息增益：当前熵与两个新群组经加权后的熵之间的差值
            p=float(len(set1))/len(rows)
            gain=current_score-p*scoref(set1)-(1-p)*scoref(set2)
            if gain>best_gain and len(set1)>0 and len(set2)>0:
                best_gain=gain
                best_criteria=(col,value)
                best_sets=(set1,set2)
    # Create the sub branches       
    if best_gain>0:
        trueBranch=buildtree(best_sets[0])
        falseBranch=buildtree(best_sets[1])
        return decisionnode(col=best_criteria[0],value=best_criteria[1],
                                                tb=trueBranch,fb=falseBranch)
    else:
        return decisionnode(results=uniquecounts(rows))


In [14]:
tree = buildtree(my_data)

### 树的显示
tree为我们得到的一棵决策树，下一步即是树的浏览：

In [15]:
def printtree(tree, indent=''):
    # Is this a leaf node?
    if tree.results != None:
            print(str(tree.results))
    else:
            # Print the criteria
            print(str(tree.col)+':'+str(tree.value)+'? ')
            # Print the branches
            print(indent+'T->', end=' ')
            printtree(tree.tb, indent+'  ')
            print(indent+'F->', end=' ')
            printtree(tree.fb, indent+'  ')

In [16]:
printtree(tree)

0:google? 
T-> 3:21? 
  T-> {'Premium': 3}
  F-> 2:no? 
    T-> {'None': 1}
    F-> {'Basic': 1}
F-> 0:slashdot? 
  T-> {'None': 3}
  F-> 2:yes? 
    T-> {'Basic': 4}
    F-> 3:21? 
      T-> {'Basic': 1}
      F-> {'None': 3}


上面的printtree以文本的形式显示, 随着树的规模逐渐变大，以这样的可视化形式来追踪我们在树上所走的路径可能是非常困难。此外，我们以一种图形化表现形式来展示树。

In [17]:
def getwidth(tree):
    if tree.tb == None and tree.fb == None: return 1
    return getwidth(tree.tb)+getwidth(tree.fb)


def getdepth(tree):
    if tree.tb == None and tree.fb == None: return 0
    return max(getdepth(tree.tb), getdepth(tree.fb))+1


In [18]:
from PIL import Image, ImageDraw

def drawtree(tree, jpeg='tree.jpg'):
    """为待绘制的树确定一个合理的尺寸，并设置好画布(canvas)，然后将画布和树的根节点传递给drawnode"""
    w = getwidth(tree)*100
    h = getdepth(tree)*100+120

    img = Image.new('RGB', (w, h), (255, 255, 255))
    canvas = ImageDraw.Draw(img)

    drawnode(canvas, tree, w/2, 20)
    img.save(jpeg, 'JPEG')

In [19]:
def drawnode(canvas, tree, x, y):
    if tree.results == None:
        # Get the width of each branch
        w1 = getwidth(tree.fb)*100
        w2 = getwidth(tree.tb)*100

        # Determine the total space required by this node
        left = x-(w1+w2)/2
        right = x+(w1+w2)/2

        # Draw the condition string
        canvas.text((x-20, y-10), str(tree.col)+':'+str(tree.value), (0, 0, 0))

        # Draw links to the branches
        canvas.line((x, y, left+w1/2, y+100), fill=(255, 0, 0))
        canvas.line((x, y, right-w2/2, y+100), fill=(255, 0, 0))

        # Draw the branch nodes
        drawnode(canvas, tree.fb, left+w1/2, y+100)
        drawnode(canvas, tree.tb, right-w2/2, y+100)
    else:
        txt = ' \n'.join(['%s:%d' % v for v in tree.results.items()])
        canvas.text((x-20, y), txt, (0, 0, 0))

In [20]:
drawtree(tree, jpeg='../datasets/treeview.jpg')

结果如下:![treeview.jpg](../datasets/treeview.jpg)

### 对新的观测数据进行分类

In [21]:
def classify(observation, tree):
    """该函数采用与printtree完全相同的方式对树进行遍历。在每次调用之后，函数会根据调用结果来判断是否到达分支的末端。"""
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        branch = None
        if isinstance(v, int) or isinstance(v, float):
            if v >= tree.value: branch = tree.tb
            else: branch = tree.fb
        else:
            if v == tree.value: branch = tree.tb
            else: branch = tree.fb
        return classify(observation, branch)

现在，依据前面已经构造的tree来预测一个新的观测数据:

In [22]:
classify(['google', 'China', 'yes', 23], tree) 

{'Premium': 3}

至此，我们已经能够从任何数据集中构造决策树，显示和解释决策树的函数，以及对新的数据进行预测的函数。但上述代码仍有缺陷: 过拟合(overfitted)

### 决策树的剪枝
前述方法可能会有过拟合的问题--也就是说，它可能变得过于针对训练数据。因为前述算法直到无法再近一步降低熵的时候才会停止分支的创建过程，所以一种可能的解决方法是：**只要当熵减少的数量小于某个阈值时就停止分支的创建。**

这种策略时常被人们采用，但是它有一个小小的缺陷--我们有可能会遇到这样的数据集: 某一次分支的创建并不会令熵降低多少，但是随后创建的分支却会使熵大幅降低。对此，一种替代的策略是，先构造好如前所述的整棵树，然后再尝试消除多余的节点--**剪枝**。

**剪枝的过程就是对具有相同父节点的一组节点进行检查，判断如果将其合并，熵的增加量是否会小于某个指定的阈值。如果是，则说明这些节点合并后对混杂度的影响在‘可接受’范围内，将会并合并成一个单一的节点。**

In [23]:
def prune(tree, mingain):
    # If the branches aren't leaves, then prune them
    if tree.tb.results == None:
        prune(tree.tb, mingain)
    if tree.fb.results == None:
        prune(tree.fb, mingain)

    # If both the subbranches are now leaves, see if they
    # should merged
    if tree.tb.results != None and tree.fb.results != None:
        # Build a combined dataset
        tb, fb = [], []
        for v, c in tree.tb.results.items():
            tb += [[v]]*c
        for v, c in tree.fb.results.items():
            fb += [[v]]*c

        # Test the reduction in entropy
        delta = entropy(tb+fb)-(entropy(tb)+entropy(fb)/2)

        if delta < mingain:
            # Merge the branches
            tree.tb, tree.fb = None, None
            tree.results = uniquecounts(tb+fb)


针对当前的数据，看看剪枝后的效果：(这个例子中，数据的拆分很容易，只要将最小增益值调得非常高的时候，某个节点才会被合并)

In [24]:
prune(tree, 0.9)
drawtree(tree, jpeg='../datasets/pruned_treeview.jpg')

<img scr='pruned_treeview.jpg'>

<img scr='pruned_treeview.jpg'>

![pruned_treeview.jpg](../datasets/pruned_treeview.jpg)

此外，决策树还有一个有点，就是它处理缺失数据的能力。我们所使用的数据集也许会缺失某些信息--比如，用户的地理位置未必能够从其IP地址中识别出来。

如果缺失了某些数据，而这些数据是确定是否分支走向所必需的，那么实际上我们可以选择两个分支都走。不过，我们不是平均地统计各分支对应的结果值，而是对其进行加权统计。

为了实现上诉功能，mdclassify是对classify的一个简单修改。

In [25]:
def mdclassify(observation, tree):
    if tree.results != None:
        return tree.results
    else:
        v = observation[tree.col]
        if v == None:
            tr, fr = mdclassify(observation, tree.tb), mdclassify(
                    observation, tree.fb)
            tcount = sum(tr.values())
            fcount = sum(fr.values())
            tw = float(tcount)/(tcount+fcount)
            fw = float(fcount)/(tcount+fcount)
            result = {}
            for k, v in tr.items(): result[k] = v*tw
            for k, v in fr.items(): result[k] = v*fw
            return result
        else:
            if isinstance(v, int) or isinstance(v, float):
                if v >= tree.value: branch = tree.tb
                else: branch = tree.fb
            else:
                if v == tree.value: branch = tree.tb
                else: branch = tree.fb
            return mdclassify(observation, branch)


mdclassify 与 classify 相比， 唯一的区别在于if v == None 那里: 如果发现有重要数据缺失，则每个分支的对应结果值都会被计算一遍，并在最终的结果值乘于它们各自的权重。

In [26]:
mdclassify(['google', None, 'yes', None], tree)

{'Premium': 2.25, 'Basic': 0.25}

In [27]:
mdclassify(['google', 'France', None, None], tree)

{'Premium': 2.25, 'None': 0.125, 'Basic': 0.125}

### 什么时候使用决策树

或许决策树的最大优势在于它可以轻易地对一个受训模型给予解释。

此外，与其它几种机器学习算法不同，决策树可以同时接受分类(categorical)数据和数值(numerical)数据作为输入。不仅如此，许多算法在运行之前都要求我们必须对输入数据做预处理，或是归一化处理，而本例中的代码却可以接受包括分类数据和数值数据在内的任何数据列表。

决策树还允许数据的不确定性分配(mdclassify允许我们使用'None'来表示一个值的缺失)，对数值型数据而言，其结果也许会落在某个已知的范围内。

不过，决策树算法还是有缺陷的。虽然对于只包含少数几种可能结果的问题而言，算法处理起来非常有效，但当面对大量可能结果的数据集时，算法就变得不那么有效了。

前述的决策树还有一个较大的缺陷，尽管它可以处理简单的数值类型数据，但是它只能创建满足“>/<”条件的节点。对于某些数据，决定分类的因素往往取决于更多变量的复杂组合，例如，假设结果值是由两个变量的差来决定的，那么这棵树就会变得非常庞大，而且预测的准确性也会迅速下降。

**综上，对于有大量数值型输入和输出的问题，决策树未必是一个好的选择；如果数值型输入之间存在许多错综复杂的关系，决策树也不是一个好的选择。**

**决策树最适合用来处理的是那些带分界点(breakpoints)的、由大量分类数据和数值数据共同组成的数据集。如果对决策过程的理解至关重要，那么采用决策树就再合适不过了。**